# Word embedding -- векторное представление слов 

В этом блокноте представлены самые базовые способы представления слов в качестве векторов.




## Подключение библиотек

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import pickle
from nltk.corpus import stopwords as sw

## Загрузка файла с корпусом текстов

Загрузка проводится из `.pkl` файла для сохранения исходных типов данных

In [2]:
with open('../serialized/dataset.pkl', 'rb') as input:
  csv = pickle.load(input)

In [3]:
csv = csv[csv.raw_json != {'description': '404 Page Not Found'}]

## Bag of words

Рассмотрим самый простой способ приведения текста к набору чисел. Для каждого слова посчитаем, как часто оно встречается в тексте. Результаты запишем в таблицу. Строки будут представлять тексты, столбцы -- слова. Если на пересечении строки с столбца стоит число 5, значит данное слово встретилось в данном тексте 5 раз. В большинстве ячеек будут нули. Поэтому хранить это всё удобнее в виде разреженных матриц (т.е. хранить только ненулевые значения).

Таким образом, при построении "мешка слов" можно выделить следующие действия:

1. Токенизация.

2. Построение словаря: собираем все слова, которые встречались в текстах и пронумеровываем их (по алфавиту, например).

3. Построение разреженной матрицы. В sklearn алгоритм приведения текста в bag-of-words реализован в виде класса `CountVectorizer`. 


В прошлом задании не были удалены некоторые стоп-слова, например, английские союзы и предлоги. Их можно "отсеять" на этапе создания модели, занеся стоп-слова в отдельный список

In [4]:
stopwords = sw.words('english')

In [5]:
count_vectorizer = CountVectorizer(stop_words=stopwords)

texts = []
for elem in csv.description:
  arr = []
  for sent in elem:
    arr.append(' '.join(word for word in sent).strip())   # Объединение всех слов из вакансии в одно предложение
  texts.append(' '.join(sent for sent in arr).strip())    # Объединение всех предложений из вакансии в один список

bow = count_vectorizer.fit_transform(texts)
print(bow.shape)
print("Число элементов в мешке слов равно числу вакансий? -",
      csv.shape[0] == bow.shape[0])

(2983, 17410)
Число элементов в мешке слов равно числу вакансий? - True


Результат содержит 2983 строки (для 2983 вакансий) и 17505 столбцов (для 17505 разных слов). Словарь:

In [6]:
count_vectorizer.vocabulary_

{'обязанность': 10960,
 'проектирование': 13063,
 'технологический': 15599,
 'раздел': 13454,
 'объект': 10939,
 'обустройство': 10898,
 'нефтяной': 10617,
 'газовый': 6383,
 'месторождение': 9794,
 'магистральный': 9508,
 'нефтегазопровод': 10606,
 'нпс': 10738,
 'кс': 9125,
 'разработка': 13483,
 'стадия': 15025,
 'согласно': 14713,
 '87': 511,
 'постановление': 12483,
 'монтажный': 10043,
 'чертеж': 16860,
 'ола': 11082,
 'тт': 15880,
 'схема': 15322,
 'регламент': 13669,
 'спецификация': 14893,
 'требование': 15798,
 'знание': 7830,
 'программа': 13002,
 'hyysis': 2180,
 'autocad': 802,
 'start': 3877,
 'microsoft': 2748,
 'office': 2940,
 'желательно': 7423,
 'pipsim': 3157,
 'гидросистема': 6507,
 'трубопровод': 15850,
 'сплит': 14928,
 'опыт': 11193,
 'пять': 13365,
 'год': 6564,
 'должность': 7162,
 'гл': 6530,
 'специалист': 14887,
 'условие': 16187,
 '40': 362,
 'часовой': 16817,
 'раб': 13374,
 'неделя': 10446,
 'тк': 15650,
 'работа': 13376,
 'адресный': 4676,
 'система': 1

In [7]:
bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 1, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [2, 0, 0, ..., 0, 0, 0]], dtype=int64)

Сохранение начальной модели:

In [8]:
dumps = pickle.dumps(count_vectorizer)

with open("../serialized/bow-first.pkl", "wb") as output:
  pickle.dump(dumps, output)

**Параметр min_df**

Помимо лемматизации/стемминга и удаления стоп-слов есть и другие способы отсечения лишнего. Например, можно откидывать слова, которые встречаются слишком редко, с помощью параметра min_df. Установив `min_df=2` мы откинем, все слова, которые встречаются менее, чем в 2 документах.

In [9]:
count_vectorizer = CountVectorizer(min_df=2)
bow = count_vectorizer.fit_transform(texts)
len(count_vectorizer.vocabulary_), count_vectorizer.vocabulary_

(8951,
 {'обязанность': 5244,
  'проектирование': 6512,
  'технологический': 7960,
  'раздел': 6734,
  'объект': 5231,
  'обустройство': 5206,
  'нефтяной': 5050,
  'газовый': 2772,
  'месторождение': 4616,
  'магистральный': 4468,
  'кс': 4266,
  'разработка': 6755,
  'стадия': 7641,
  'согласно': 7457,
  '87': 220,
  'постановление': 6145,
  'монтажный': 4741,
  'чертеж': 8663,
  'ола': 5319,
  'тт': 8110,
  'схема': 7809,
  'регламент': 6873,
  'спецификация': 7568,
  'требование': 8065,
  'знание': 3548,
  'программа': 6474,
  'autocad': 343,
  'start': 1553,
  'microsoft': 1086,
  'office': 1162,
  'желательно': 3309,
  'трубопровод': 8089,
  'опыт': 5381,
  'пять': 6680,
  'год': 2858,
  'должность': 3174,
  'гл': 2843,
  'специалист': 7562,
  'условие': 8302,
  '40': 157,
  'часовой': 8636,
  'раб': 6685,
  'неделя': 4964,
  'тк': 7982,
  'работа': 6686,
  'адресный': 1890,
  'система': 7296,
  'склад': 7321,
  'набор': 4808,
  'товар': 7989,
  'комплектовка': 4039,
  'автозапча

Размер словаря уменьшился с 17505 слов до 8951 слов

Сохранение модели с отсечением редких слов:

In [10]:
dumps = pickle.dumps(count_vectorizer)

with open("../serialized/bow-mindf.pkl", "wb") as output:
  pickle.dump(dumps, output)

**Биграммы, триграммы, n-граммы**

По умолчанию bag-of-words (как следует из названия) представляет собой просто мешок слов. То есть для него предложения "It's not good, it's bad!" и "It's not bad, it's good!" абсолютно эквивалентны. Понятно, что при этом теряется много информации. Можно рассматривать не только отдельные слова, а последовательности длиной из 2 слов (биграммы), из 3 слов (триграммы) или в общем случае из n слов (n-граммы). На практике обычно задаётся диапазон от 1 до n.

In [11]:
count_vectorizer = CountVectorizer(ngram_range=(1,5), min_df=500)
bow = count_vectorizer.fit_transform(texts)
print(sorted(count_vectorizer.vocabulary_))

df = pd.DataFrame(bow.toarray(),
                     columns=count_vectorizer.get_feature_names())
display(df)

['00', 'ведение', 'весь', 'возможность', 'выполнение', 'высокий', 'год', 'график', 'график работа', 'день', 'документация', 'заработный', 'заработный плата', 'знание', 'карьерный', 'карьерный рост', 'клиент', 'коллектив', 'команда', 'компания', 'контроль', 'корпоративный', 'место', 'месяц', 'навык', 'наличие', 'наш', 'новый', 'образование', 'обучение', 'обязанность', 'оклад', 'оплата', 'опыт', 'опыт работа', 'организация', 'ответственность', 'офис', 'официальный', 'оформление', 'оформление тк', 'пакет', 'пк', 'плата', 'полный', 'пользователь', 'предлагать', 'проведение', 'продажа', 'профессиональный', 'работа', 'работать', 'рабочий', 'развитие', 'результат', 'рост', 'рф', 'свой', 'система', 'соответствие', 'сотрудник', 'тк', 'тк рф', 'требование', 'требование высокий', 'требование опыт', 'трудоустройство', 'умение', 'уровень', 'условие', 'участие']


,00,ведение,весь,возможность,выполнение,высокий,год,график,график работа,день,...,тк,тк рф,требование,требование высокий,требование опыт,трудоустройство,умение,уровень,условие,участие
0,0,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2,0,0,0,2,0,0,0,1,1,1,...,0,0,1,0,0,0,0,0,1,0
3,0,0,0,0,1,0,0,1,1,0,...,1,1,1,0,1,0,1,0,1,0
4,0,1,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,0,0,0,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
2979,0,1,1,0,1,1,0,0,0,0,...,1,1,2,1,0,0,0,1,2,0
2980,0,0,1,2,0,0,2,1,0,3,...,1,1,0,0,0,0,0,0,0,0
2981,0,0,0,0,0,1,0,2,2,0,...,1,1,2,1,0,0,0,0,2,1


В параметрах `CountVectorizer` был передан размер энграммы вплоть до 5 слов, однако на выходе получается последовательность из максимум 2 слов:

  - 'заработный плата'
  - 'карьерный рост'
  - 'опыт работа'
  - 'оформление тк'
  - 'тк рф'
  - 'требование высокий'
  - 'требование опыт'

Это связано с тем, что энграммы большей длины находятся в числе документов меньшем, чем указано в `min_df` (500)

Сохранение модели с энграммами:

In [12]:
dumps = pickle.dumps(count_vectorizer)

with open("../serialized/bow-ngrams.pkl", "wb") as output:
  pickle.dump(dumps, output)

**Ограничение количества признаков**

Понятно, что с ростом n количество выделенных n-грамм быстро растёт. Для ограничения количества признаков можно использовать параметр max_features. В этом случае будет создано не более max_features признаков (будут выбраны самые часто встречающиеся слова и последовательности слов). Например:

In [12]:
count_vectorizer = CountVectorizer(ngram_range=(1,2), max_features=25)
bow = count_vectorizer.fit_transform(texts)
count_vectorizer.vocabulary_

{'обязанность': 13,
 'требование': 23,
 'знание': 7,
 'опыт': 14,
 'год': 2,
 'условие': 24,
 'тк': 22,
 'работа': 19,
 'график': 3,
 'обучение': 12,
 'график работа': 4,
 'продажа': 18,
 'клиент': 8,
 'оформление': 16,
 'рф': 21,
 'заработный': 5,
 'плата': 17,
 'опыт работа': 15,
 'заработный плата': 6,
 'компания': 9,
 'образование': 11,
 'контроль': 10,
 'работать': 20,
 'высокий': 1,
 '00': 0}

Сохранение модели c ограниченным вокабуляром:

In [13]:
dumps = pickle.dumps(count_vectorizer)

with open("../serialized/bow-maxfeat.pkl", "wb") as output:
  pickle.dump(dumps, output)

## TF-IDF

У подхода bag-of-words есть существенный недостаток. Если слово встречается 5 раз в конкретном документе, но и в других документах тоже встречается часто, то его наличие в документе не особо-то о чём-то говорит. Если же слово 5 раз встречается в конкретном документе, но в других документах встречается редко, то его наличие (да ещё и многократное) позволяет хорошо отличать этот документ от других. Однако с точки зрения bag-of-words различий не будет: в обеих ячейках будет просто число 5.

Отчасти это решается исключением стоп-слов (и слишком часто встречающихся слов), но лишь отчасти. Другой идеей является отмасштабировать получившуюся таблицу с учётом "редкости" слова в наборе документов (т.е. с учётом информативности слова).

$$tfidf=tf∗idf$$

$$idf=\log\frac{(N+1)}{(Nw+1)}+1$$

Здесь tf это частота слова в тексте (то же самое, что в bag of words), N - общее число документов, Nw - число документов, содержащих данное слово.

То есть для каждого слова считается отношение общего количества документов к количеству документов, содержащих данное слово (для частых слов оно будет ближе к 1, для редких слов оно будет стремиться к числу, равному количеству документов), и на логарифм от этого числа умножается исходное значение bag-of-words (к числителю и знаменателю прибавляется единичка, чтобы не делить на 0, и к логарифму тоже прибавляется единичка, но это уже технические детали). После этого в sklearn ещё проводится L2-нормализация каждой строки.

В sklearn есть класс для поддержки TF-IDF: `TfidfVectorizer`, рассмотрим его.

In [14]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(texts)
len(tfidf_vectorizer.vocabulary_), tfidf_vectorizer.vocabulary_

(17505,
 {'обязанность': 11055,
  'проектирование': 13158,
  'технологический': 15694,
  'раздел': 13549,
  'объект': 11034,
  'обустройство': 10993,
  'нефтяной': 10712,
  'газовый': 6478,
  'месторождение': 9889,
  'магистральный': 9603,
  'нефтегазопровод': 10701,
  'нпс': 10833,
  'кс': 9220,
  'разработка': 13578,
  'стадия': 15120,
  'согласно': 14808,
  '87': 511,
  'постановление': 12578,
  'монтажный': 10138,
  'чертеж': 16955,
  'ола': 11177,
  'тт': 15975,
  'схема': 15417,
  'регламент': 13764,
  'спецификация': 14988,
  'требование': 15893,
  'знание': 7925,
  'программа': 13097,
  'hyysis': 2214,
  'autocad': 814,
  'start': 3941,
  'microsoft': 2791,
  'office': 2991,
  'желательно': 7518,
  'pipsim': 3216,
  'гидросистема': 6602,
  'трубопровод': 15945,
  'сплит': 15023,
  'опыт': 11288,
  'пять': 13460,
  'год': 6659,
  'должность': 7257,
  'гл': 6625,
  'специалист': 14982,
  'условие': 16282,
  '40': 362,
  'часовой': 16912,
  'раб': 13469,
  'неделя': 10541,
  'тк':

Словарь содержит те же 17505 значений, которые были бы и для изначального `CountVectorizer`. Но значения в таблице другие:

In [15]:
tfidf.todense()

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.14649537, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.04251854, 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.05477631, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

Ненулевые значения находятся на тех же местах, но отмасштабированы в зависимости от частоты слов.

In [16]:
df = pd.DataFrame(tfidf.toarray(),
             columns=tfidf_vectorizer.get_feature_names())
df.head(25)

,00,000,000р,000руб,0015,002,003,004,00или,00ч,...,ідеї,із,ін,іногородніх,інструменти,інструментів,інших,їзд,їхньому,құрылыс
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.146495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.094925,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.136446,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Сохранение `TfIdf`-модели:

In [17]:
dumps = pickle.dumps(tfidf_vectorizer)

with open("../serialized/tfidf.pkl", "wb") as output:
  pickle.dump(dumps, output)

**Параметр sublinear_tf**

Большая часть параметров у `CountVectorizer` и `TfidfVectorizer` одинакова. Но у `TfidfVectorizer` есть один важный дополнительный параметр.

Как видно из формулы $tfidf = tf \cdot idf$, если слово будет встречаться не один, а два раза, то `tfidf` вырастет в два раза. Если слово будет встречаться не один, а 10 раз, то `tfidf` вырастет почти в 10 раз.

In [18]:
top_features = df.max().sort_values(ascending=False).head(15).index
display(df[top_features].max().sort_values(ascending=False))

игровой                0.841335
медиа                  0.807344
туристский             0.803411
кладка                 0.794022
электрооборудование    0.761479
эпу                    0.760127
агент                  0.738356
ультразвуковой         0.729910
недвижимость           0.721568
тц                     0.719580
клуб                   0.709043
экспортный             0.708224
бетонный               0.690345
кс                     0.686658
проект                 0.684626
dtype: float64

Если необходимо уменьшить рост значения `tfidf` с числом появлений, то можно использовать параметр `sublinear_tf=True`. При его использовании вместо `tf` будет браться $1 + log(tf)$. То есть по-прежнему с ростом `tf` будет расти и `tfidf`, но уже не так радикально (и соответственно остальные значения будут уменьшаться не так быстро). Для некоторых задач это может дать прирост в качестве.

In [19]:
tfidf_vectorizer = TfidfVectorizer(sublinear_tf=True)
tfidf = tfidf_vectorizer.fit_transform(texts)
df = pd.DataFrame(tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names())
display(df[top_features].max().sort_values(ascending=False))

ультразвуковой         0.670602
кладка                 0.567360
электрооборудование    0.537308
экспортный             0.519852
эпу                    0.509715
игровой                0.469216
кс                     0.464843
бетонный               0.421728
туристский             0.418830
агент                  0.417142
медиа                  0.413230
недвижимость           0.334231
проект                 0.328216
тц                     0.320601
клуб                   0.310729
dtype: float64

Как можно видеть, значение tfidf для слова **"игровой"** уменьшилось почти в два раза (с 0.841335 до 0.469216), а оценка слова **"ультразвуковой"** упала не так сильно, как у других, за счёт этого оно имеет наивысшую оценку при использовании параметра `sublinear_tf=True`

Сохранение `TfIdf`-модели с параметром `sublinear=True`:

In [20]:
dumps = pickle.dumps(tfidf_vectorizer)

with open("../serialized/tfidf-sublinear.pkl", "wb") as output:
  pickle.dump(dumps, output)

## Добавление столбцов с результатами преобразований

In [21]:
# with open('../serialized/bow-first.pkl', 'rb') as input:
#   count_vectorizer = pickle.loads(pickle.load(input))
# 
# with open('../serialized/tfidf.pkl', 'rb') as input:
#   tfidf_vectorizer = pickle.loads(pickle.load(input))

count_vectorizer = CountVectorizer(ngram_range=(1, 3), min_df=200)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), min_df=200)

bow = count_vectorizer.fit_transform(texts).toarray()
tfidf = tfidf_vectorizer.fit_transform(texts).toarray()

print(count_vectorizer.vocabulary_)
print(len(count_vectorizer.vocabulary_))
print(count_vectorizer.vocabulary_ == tfidf_vectorizer.vocabulary_)

{'обязанность': 126, 'объект': 124, 'разработка': 199, 'согласно': 224, 'требование': 252, 'знание': 73, 'программа': 180, 'office': 11, 'желательно': 66, 'опыт': 132, 'год': 42, 'должность': 58, 'специалист': 233, 'условие': 268, 'неделя': 113, 'тк': 248, 'работа': 190, 'система': 217, 'склад': 219, 'товар': 250, 'производство': 185, 'рабочий': 194, '000': 3, 'результат': 203, 'собеседование': 221, 'рабочий неделя': 196, 'обслуживание': 122, 'покупатель': 163, 'расчет': 200, 'срок': 237, 'наличие': 110, 'ответственность': 138, 'внимательность': 31, 'график': 48, 'возможный': 34, 'день': 52, 'возможность': 32, 'социальный': 231, 'оплата': 130, 'отпуск': 140, 'больничный': 21, 'дом': 59, 'скидка': 218, 'магазин': 97, 'сеть': 216, 'профессиональный': 186, 'карьерный': 81, 'рост': 207, 'корпоративный': 93, 'обучение': 123, 'мероприятие': 101, 'условие график': 269, 'график работа': 49, 'рабочий день': 195, 'возможность профессиональный': 33, 'профессиональный карьерный': 187, 'карьерный р

In [22]:
bow_column, tfidf_column = [], []
for i in range(len(csv)):
  bow_column.append(bow[i])
  tfidf_column.append(tfidf[i])

new_csv = csv.copy()
new_csv['bag_of_words'], new_csv['tfidf'] = bow_column, tfidf_column
new_csv

,id,raw_json,description,bag_of_words,tfidf
0,20514076,"{'id': '20514076', 'premium': False, 'billing_...","[[обязанность, проектирование, технологический...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,27771233,"{'id': '27771233', 'premium': False, 'billing_...","[[обязанность, работа, адресный, система, скла...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.3232301568297295, 0.0, 0.0, ..."
2,36743137,"{'id': '36743137', 'premium': False, 'billing_...","[[обязанность, обслуживание, покупатель, касса...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,16812757,"{'id': '16812757', 'premium': False, 'billing_...","[[обязанность, оптимизация, работа, существующ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, ...","[0.0, 0.0, 0.0, 0.14538412492839456, 0.0, 0.0,..."
4,41806844,"{'id': '41806844', 'premium': False, 'billing_...","[[обязанность, продвижение, продукт, компания,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...
3495,29938082,"{'id': '29938082', 'premium': False, 'billing_...","[[обязанность, обслуживание, электрооборудован...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3496,11330116,"{'id': '11330116', 'premium': False, 'billing_...","[[обязанность, организация, координация, склад...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3497,40856973,"{'id': '40856973', 'premium': False, 'billing_...","[[скучный, работа, мало, платить, давать, разв...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.08090344059183333, 0.0, 0.0,..."
3498,25193210,"{'id': '25193210', 'premium': False, 'billing_...","[[обязанность, квалифицировать, своевременный,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [23]:
new_csv.to_pickle('../serialized/new_dataset.pkl')
new_csv.to_csv('../csv/result_2.csv')

## Задание

1. Создать CountVectorizer и TfidfVectorizer для ваших корпусов документов.
2. Сериализовать и сохранить на диск эти модели.
3. Дописать в csv два дополнительных столбца с результатами преобразования с помощью двух методов.


## Анализ набора текстов

С помощью мешка слов и его преобразования в `DataFrame` можно провести анализ набора текстов: узнать, сколько раз появляется определённое слово в документе, какое слово встречается чаще всего в одном документе, общее число появлений слово во всём корпусе текстов и т.д

In [24]:
with open('../serialized/bow-first.pkl', 'rb') as input:
  cv = pickle.loads(pickle.load(input))

bow = cv.transform(texts) 

df = pd.DataFrame(bow.toarray(), columns=cv.get_feature_names())
display(df)

,00,000,000р,000руб,0015,002,003,004,00или,00ч,...,ідеї,із,ін,іногородніх,інструменти,інструментів,інших,їзд,їхньому,құрылыс
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2979,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2980,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2981,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Наибольшее повторение слова в одном документе:

In [25]:
df.max().sort_values(ascending=False).head(25)

технический        25
procurement        21
работа             21
обслуживание       20
оборудование       20
продукция          19
контракт           19
ребенок            18
обучение           17
предприятие        17
среда              17
больной            17
проект             17
tax                17
медицинский        16
охрана             16
труд               16
00                 16
проживать          15
контроль           15
технологический    15
недвижимость       15
свой               15
окружающий         15
объект             14
dtype: int64

In [26]:
df['технический'].sort_values(ascending=False)

894     25
649      8
636      7
980      7
530      7
        ..
1086     0
1087     0
1088     0
1089     0
1491     0
Name: технический, Length: 2983, dtype: int64

Слово **"технический"** имеет наибольшее число появлений в одном тексте (под номером 894)

In [27]:
texts[894]

'цель должность обязательный роль основной функция технический целостность проект соблюдение график выполнение весь инженерно-технический задача проект обязанность обеспечивать необходимый выполнение цель проект технический руководство обеспечивать управление качество конфигурация стоимость технический инжиниринг псд проект выступать качество советник технический вопрос руководитель проект функциональный подразделение указание руководитель проект анализировать согласовывать весь затрата календарный план письменный устный отчет инженерно-технический задача подзадача получать заказчик весь необходимый технический информация также выяснять ограничение стоимость срок убеждаться технический требование заказчик правильно понимать компания способный выполнять иметься уровень технический оснащение подготовка определять требование мелкий подсистема функциональный подразделение служба возможно разрабатывать календарный план прогнозировать затрата выполнять работа должный образ сотрудничество под

Число появлений слов во всех документах:

In [28]:
df.sum().sort_values(ascending=False).head(25)

работа              9603
опыт                3442
компания            3019
условие             2848
требование          2846
обязанность         2585
знание              1962
продажа             1926
клиент              1872
график              1740
00                  1562
оформление          1547
высокий             1515
год                 1499
контроль            1405
плата               1396
заработный          1382
образование         1359
рф                  1356
обучение            1306
работать            1269
тк                  1230
возможность         1192
профессиональный    1109
рабочий             1092
dtype: int64

Слово **"работа"** появляется во всем корпусе в наибольшем количестве - 9603 раза